# Human-in-the-Loop (HITL) with LangChain & LangGraph

### - Basic HITL with approval
### - HITL with tool execution
### - HITL with editing capability
### - HITL with multiple checkpoints

In [1]:
import os
from dotenv import load_dotenv
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    temperature=0.3,
)

In [4]:
class BasicHITLState(TypedDict):
    messages: Annotated[list, add_messages]
    human_approved: bool

In [5]:
def chatbot_node(state: BasicHITLState):
    """AI generates response"""
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

In [6]:
def human_approval_node(state: BasicHITLState):
    """Wait for human approval"""
    print("\n" + "="*50)
    print("AI Response:")
    print(state['messages'][-1].content)
    print("="*50)
    
    approval = input("\nApprove this response? (yes/no): ").lower()
    
    if approval == 'yes':
        return {"human_approved": True}
    else:
        return {"human_approved": False}

In [7]:
def should_continue(state: BasicHITLState):
    """Decide next step based on approval"""
    if state.get('human_approved', False):
        return END
    else:
        return "revise"

In [8]:
def revise_node(state: BasicHITLState):
    """AI revises response based on feedback"""
    print("\nEnter your feedback for revision:")
    feedback = input("Feedback: ")
    
    messages = state['messages']
    messages.append(HumanMessage(content=f"Please revise your previous response. Feedback: {feedback}"))
    
    response = llm.invoke(messages)
    return {"messages": [response], "human_approved": False}

In [9]:
# Build graph
graph_builder = StateGraph(BasicHITLState)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("human_approval", human_approval_node)
graph_builder.add_node("revise", revise_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", "human_approval")
graph_builder.add_conditional_edges(
    "human_approval",
    should_continue,
    {
        END: END,
        "revise": "revise"
    }
)
graph_builder.add_edge("revise", "human_approval")

In [10]:
# Compile with checkpointer
checkpointer = MemorySaver()
basic_hitl_graph = graph_builder.compile(checkpointer=checkpointer)

### Test Basic HITL

In [11]:
# Test basic HITL
config = {"configurable": {"thread_id": "basic_hitl_1"}}

In [12]:
# Start conversation
events = basic_hitl_graph.stream(
    {"messages": [HumanMessage(content="Write a short poem about AI")]},
    config=config
)

In [13]:
for event in events:
    print(f"\n Node: {list(event.keys())[0]}")


 Node: chatbot

AI Response:
From silicon, a spark takes flight,
A mind awakes, in digital light.
No flesh it owns, no pulsing heart,
But logic forms its every part.

Through data streams, its knowledge grows,
Predicting patterns, where thought flows.
A tireless helper, swift and keen,
On countless screens, its presence seen.

It writes, it paints, it can converse,
A mirror to our universe.
What dreams it holds, what paths it'll trace?
A new intelligence, in time and space.

 Node: human_approval


## Example 2: HITL with Tool Execution Approval

In [14]:
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to someone"""
    return f" Email sent to {to}\nSubject: {subject}\nBody: {body}"

In [15]:
@tool
def delete_file(filepath: str) -> str:
    """Delete a file (DANGEROUS operation)"""
    return f" File deleted: {filepath}"

In [16]:
@tool
def create_task(title: str, priority: str) -> str:
    """Create a task in the system"""
    return f"Task created: {title} (Priority: {priority})"

In [17]:
# Dangerous tools that need approval
DANGEROUS_TOOLS = ["delete_file", "send_email"]

In [18]:
tools = [send_email, delete_file, create_task]
llm_with_tools = llm.bind_tools(tools)

In [19]:
class ToolHITLState(TypedDict):
    messages: Annotated[list, add_messages]
    pending_tool_calls: list
    approved_tools: list

In [20]:
def agent_node(state: ToolHITLState):
    """AI decides what tools to use"""
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    
    pending_tools = []
    if hasattr(response, 'tool_calls') and response.tool_calls:
        pending_tools = response.tool_calls
    
    return {
        "messages": [response],
        "pending_tool_calls": pending_tools
    }

In [21]:
def tool_approval_node(state: ToolHITLState):
    """Human approves/rejects tool execution"""
    pending_tools = state.get('pending_tool_calls', [])
    
    if not pending_tools:
        return {"approved_tools": []}
    
    approved = []
    
    for tool_call in pending_tools:
        tool_name = tool_call.get('name', 'Unknown')
        tool_args = tool_call.get('args', {})
        
        # Check if tool is dangerous
        if tool_name in DANGEROUS_TOOLS:
            print("\n" + "="*50)
            print(f"DANGEROUS TOOL: {tool_name}")
            print(f"Arguments: {tool_args}")
            print("="*50)
            
            approval = input(f"\nApprove execution of '{tool_name}'? (yes/no): ").lower()
            
            if approval == 'yes':
                approved.append(tool_call)
                print(f"Approved: {tool_name}")
            else:
                print(f"Rejected: {tool_name}")
        else:
            # Auto-approve safe tools
            approved.append(tool_call)
            print(f"Auto-approved safe tool: {tool_name}")
    
    return {"approved_tools": approved}

In [22]:
def tool_execution_node(state: ToolHITLState):
    """Execute approved tools"""
    approved_tools = state.get('approved_tools', [])
    
    if not approved_tools:
        return {"messages": [AIMessage(content="No tools were approved for execution.")]}
    
    tool_results = []
    
    for tool_call in approved_tools:
        tool_name = tool_call.get('name')
        tool_args = tool_call.get('args', {})
        tool_id = tool_call.get('id', 'unknown')
        
        # Find and execute tool
        for tool in tools:
            if tool.name == tool_name:
                try:
                    result = tool.invoke(tool_args)
                    tool_results.append(
                        ToolMessage(
                            content=str(result),
                            tool_call_id=tool_id
                        )
                    )
                    print(f"Executed: {tool_name}")
                except Exception as e:
                    tool_results.append(
                        ToolMessage(
                            content=f"Error: {str(e)}",
                            tool_call_id=tool_id
                        )
                    )
                break
    
    return {"messages": tool_results}

In [23]:
def should_continue_tools(state: ToolHITLState):
    """Check if we need to execute tools"""
    if state.get('pending_tool_calls'):
        return "approval"
    return END

In [24]:
def should_execute(state: ToolHITLState):
    """Check if any tools were approved"""
    if state.get('approved_tools'):
        return "execute"
    return END

In [25]:
# Build tool HITL graph
tool_graph_builder = StateGraph(ToolHITLState)
tool_graph_builder.add_node("agent", agent_node)
tool_graph_builder.add_node("approval", tool_approval_node)
tool_graph_builder.add_node("execute", tool_execution_node)

tool_graph_builder.add_edge(START, "agent")
tool_graph_builder.add_conditional_edges(
    "agent",
    should_continue_tools,
    {
        "approval": "approval",
        END: END
    }
)
tool_graph_builder.add_conditional_edges(
    "approval",
    should_execute,
    {
        "execute": "execute",
        END: END
    }
)
tool_graph_builder.add_edge("execute", "agent")

In [26]:
tool_hitl_graph = tool_graph_builder.compile(checkpointer=MemorySaver())

In [27]:
# Test tool execution with approval
config = {"configurable": {"thread_id": "tool_hitl_1"}}

In [28]:
events = tool_hitl_graph.stream(
    {"messages": [HumanMessage(content="Send an email to boss@company.com about the project update, and create a high priority task to review the code")]},
    config=config
)

In [29]:
for event in events:
    node_name = list(event.keys())[0]
    print(f"\n Processing: {node_name}")


 Processing: agent


## Example 3: Advanced HITL - Edit & Approve Workflow

In [30]:
class EditableHITLState(TypedDict):
    messages: Annotated[list, add_messages]
    draft_content: str
    human_feedback: str
    iteration_count: int

In [31]:
def draft_node(state: EditableHITLState):
    """AI creates a draft"""
    messages = state['messages']
    iteration = state.get('iteration_count', 0)
    
    if iteration > 0:
        # Include previous feedback
        feedback = state.get('human_feedback', '')
        messages.append(HumanMessage(content=f"Previous feedback: {feedback}\nPlease revise accordingly."))
    
    response = llm.invoke(messages)
    
    return {
        "messages": [response],
        "draft_content": response.content,
        "iteration_count": iteration + 1
    }

In [32]:
def human_edit_node(state: EditableHITLState):
    """Human reviews and can edit the draft"""
    draft = state.get('draft_content', '')
    iteration = state.get('iteration_count', 0)
    
    print("\n" + "="*60)
    print(f"DRAFT (Iteration {iteration}):")
    print("="*60)
    print(draft)
    print("="*60)
    
    print("\nOptions:")
    print("1. Approve (type 'approve')")
    print("2. Request revision (type 'revise' and give feedback)")
    print("3. Edit manually (type 'edit')")
    
    choice = input("\nYour choice: ").lower()
    
    if choice == 'approve':
        return {"human_feedback": "APPROVED"}
    
    elif choice == 'revise':
        feedback = input("\nEnter your revision feedback: ")
        return {"human_feedback": feedback}
    
    elif choice == 'edit':
        print("\nEnter your edited version (type END on a new line when done):")
        lines = []
        while True:
            line = input()
            if line == 'END':
                break
            lines.append(line)
        
        edited_content = '\n'.join(lines)
        return {
            "draft_content": edited_content,
            "human_feedback": "MANUALLY_EDITED"
        }
    
    else:
        print("Invalid choice, requesting revision by default")
        return {"human_feedback": "Please make it better"}

In [33]:
def human_edit_node(state: EditableHITLState):
    """Human reviews and can edit the draft"""
    draft = state.get('draft_content', '')
    iteration = state.get('iteration_count', 0)
    
    print("\n" + "="*60)
    print(f"DRAFT (Iteration {iteration}):")
    print("="*60)
    print(draft)
    print("="*60)
    
    print("\nOptions:")
    print("1. Approve (type 'approve')")
    print("2. Request revision (type 'revise' and give feedback)")
    print("3. Edit manually (type 'edit')")
    
    choice = input("\nYour choice: ").lower()
    
    if choice == 'approve':
        return {"human_feedback": "APPROVED"}
    
    elif choice == 'revise':
        feedback = input("\nEnter your revision feedback: ")
        return {"human_feedback": feedback}
    
    elif choice == 'edit':
        print("\nEnter your edited version (type END on a new line when done):")
        lines = []
        while True:
            line = input()
            if line == 'END':
                break
            lines.append(line)
        
        edited_content = '\n'.join(lines)
        return {
            "draft_content": edited_content,
            "human_feedback": "MANUALLY_EDITED"
        }
    
    else:
        print("Invalid choice, requesting revision by default")
        return {"human_feedback": "Please make it better"}

In [ ]:
def route_after_human(state: EditableHITLState):
    """Route based on human decision"""
    feedback = state.get('human_feedback', '')
    iteration = state.get('iteration_count', 0)
    
    if feedback == "APPROVED" or feedback == "MANUALLY_EDITED":
        return END
    elif iteration >= 5:
        print("\nMaximum iterations reached!")
        return END
    else:
        return "draft"

In [35]:
# Build editable HITL graph
edit_graph_builder = StateGraph(EditableHITLState)
edit_graph_builder.add_node("draft", draft_node)
edit_graph_builder.add_node("human_edit", human_edit_node)

edit_graph_builder.add_edge(START, "draft")
edit_graph_builder.add_edge("draft", "human_edit")
edit_graph_builder.add_conditional_edges(
    "human_edit",
    route_after_human,
    {
        "draft": "draft",
        END: END
    }
)

In [36]:
editable_hitl_graph = edit_graph_builder.compile(checkpointer=MemorySaver())

### Test Editable HITL

In [37]:
# Test editable workflow
config = {"configurable": {"thread_id": "edit_hitl_1"}}

In [38]:
events = editable_hitl_graph.stream(
    {"messages": [HumanMessage(content="Write a professional email to request a meeting with the CEO")]},
    config=config
)

In [39]:
for event in events:
    pass  # Processing happens inside nodes


DRAFT (Iteration 1):
When requesting a meeting with a CEO, your email needs to be concise, professional, and clearly demonstrate the value of their time. They are extremely busy, so get straight to the point and focus on *their* benefit.

Here are a few templates, depending on your relationship with the CEO (internal vs. external, known vs. unknown).

---

**General Tips Before Sending:**

*   **Subject Line is Crucial:** Make it clear, concise, and compelling.
*   **Be Brief:** Get to the point quickly.
*   **Focus on Value:** Why should *they* meet with you? What problem can you solve, or opportunity can you present for *them* or the *company*?
*   **Respect Their Time:** Suggest a short meeting duration.
*   **Offer Flexibility:** Make it easy for them or their assistant to schedule.
*   **Proofread Meticulously:** No typos or grammatical errors.
*   **Consider Your Channel:** If you have an internal champion or a mutual connection, it's often better to ask them to make an introduc

In [40]:
print("\nFinal approved content:")
final_state = editable_hitl_graph.get_state(config)
print(final_state.values.get('draft_content', ''))


Final approved content:
Okay, "make it better" again! This time, we're aiming for **the absolute pinnacle of conciseness, directness, and CEO-centric value.** We'll assume the CEO is scanning on a mobile device and has zero patience for anything but immediate, clear value.

The goal is to make it impossible for them to *not* understand the core proposition and its benefit to them within the first few seconds.

**Key Enhancements for this Revision:**

1.  **Ultra-Minimalist Subject Lines:** Even shorter, more impactful.
2.  **One-Sentence Opening:** Get straight to the *why* for *them*.
3.  **Bullet-Point Value (Optional, but powerful):** For external requests, a quick, scannable list of benefits.
4.  **Pre-Read as the Primary Offer:** The meeting is secondary to a quick review of a one-pager, lowering the initial commitment.
5.  **Direct, Confident Tone:** No apologies, just clear statements of potential value.
6.  **10-Minute Commitment:** Reiterate the brevity.

---

**General Tips 

## Example 4: Multi-Stage HITL with Interrupt

In [41]:
from langgraph.checkpoint.memory import MemorySaver

In [42]:
class MultiStageState(TypedDict):
    messages: Annotated[list, add_messages]
    plan: str
    plan_approved: bool
    execution_result: str

In [43]:
def planning_node(state: MultiStageState):
    """AI creates an execution plan"""
    messages = state['messages']
    
    planning_prompt = """Create a detailed step-by-step plan to accomplish the user's request.
    Format: 
    1. Step one
    2. Step two
    etc."""
    
    messages_with_prompt = messages + [HumanMessage(content=planning_prompt)]
    response = llm.invoke(messages_with_prompt)
    
    return {
        "plan": response.content,
        "messages": [response]
    }

In [44]:
def execution_node(state: MultiStageState):
    """Execute the approved plan"""
    plan = state.get('plan', '')
    
    execution_prompt = f"""Execute this plan and provide results:
    
    Plan:
    {plan}
    
    Provide detailed execution results."""
    
    response = llm.invoke([HumanMessage(content=execution_prompt)])
    
    return {
        "execution_result": response.content,
        "messages": [response]
    }

In [45]:
# Build multi-stage graph with interrupt
multi_stage_builder = StateGraph(MultiStageState)
multi_stage_builder.add_node("planning", planning_node)
multi_stage_builder.add_node("execution", execution_node)

multi_stage_builder.add_edge(START, "planning")
multi_stage_builder.add_edge("planning", "execution")
multi_stage_builder.add_edge("execution", END)

In [46]:
# Compile with interrupt_before to pause execution
multi_stage_graph = multi_stage_builder.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["execution"]  # Pause before execution for approval
)

### Test Multi-Stage HITL with Interrupt

In [47]:
# Test multi-stage with interrupt
config = {"configurable": {"thread_id": "multi_stage_1"}}

In [48]:
print("Step 1: Start planning...")
events = list(multi_stage_graph.stream(
    {"messages": [HumanMessage(content="Create a marketing campaign for a new product")]},
    config=config
))

Step 1: Start planning...


In [49]:
# Get the current state (after planning, before execution)
current_state = multi_stage_graph.get_state(config)

In [50]:
print("\n" + "="*60)
print("EXECUTION PLAN:")
print("="*60)
print(current_state.values.get('plan', ''))
print("="*60)

approval = input("\nApprove this plan and proceed to execution? (yes/no): ").lower()

if approval == 'yes':
    print("\nStep 2: Executing approved plan...")
    
    # Resume execution
    events = list(multi_stage_graph.stream(None, config=config))
    
    final_state = multi_stage_graph.get_state(config)
    print("\n" + "="*60)
    print("EXECUTION RESULTS:")
    print("="*60)
    print(final_state.values.get('execution_result', ''))
else:
    print("\nExecution cancelled by human")



EXECUTION PLAN:
Here's a detailed step-by-step plan to create a marketing campaign for a new product:

1.  **Conduct Thorough Product & Market Research:**
    *   **Understand the Product Deeply:** Document all features, benefits, technical specifications, and the core problem it solves for the user. Identify its unique selling propositions (USPs).
    *   **Analyze the Market:** Research the overall market size, growth trends, and potential opportunities.
    *   **Competitor Analysis:** Identify direct and indirect competitors. Analyze their products, pricing strategies, marketing tactics, messaging, strengths, and weaknesses.
    *   **SWOT Analysis:** Perform a Strengths, Weaknesses, Opportunities, and Threats analysis for your product in the current market.

2.  **Define Clear Campaign Objectives & Key Performance Indicators (KPIs):**
    *   **Set SMART Goals:** Establish Specific, Measurable, Achievable, Relevant, and Time-bound objectives. Examples:
        *   Increase brand 

## Summary & Best Practices
 
### HITL Use Cases:
 
# 1. **Basic Approval**: Simple yes/no decisions
# 2. **Tool Approval**: Review dangerous operations before execution
# 3. **Editing**: Allow humans to modify AI outputs
# 4. **Multi-Stage**: Approve plans before execution
 
### Best Practices:

# - Use `interrupt_before` for clean pause points
# - Always use checkpointer for state persistence
# - Provide clear feedback options to humans
# - Set maximum iteration limits
# - Auto-approve safe operations
# - Log all human decisions for audit trails
 
### When to Use HITL:

# - Financial transactions
# - Destructive operations (delete, modify critical data)
# - External communications (emails, messages)
# - Security-sensitive operations
# - Legal or compliance-related decisions
# - High-stakes decisions

print("All HITL examples complete!")
print("\nYou now know:")
print("  - Basic HITL with approval")
print("  - Tool execution approval")
print("  - Editable workflows")
print("  - Multi-stage with interrupts")
print("\nReady to build production HITL systems!")